In [ ]:
#import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load the training and testing data, scale it into the range [0, 1],
# then reshape the design matrix
print("[INFO] accessing CIFAR10...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()  # Load CIFAR-10 dataset into training and testing sets
trainX = trainX.astype("float") / 255.0                    # Scale the training data to the range [0, 1]
testX = testX.astype("float") / 255.0                      # Scale the testing data to the range [0, 1]
trainX = trainX.reshape((trainX.shape[0], 3072))           # Reshape the training data to have 3072 features (32x32x3)
testX = testX.reshape((testX.shape[0], 3072))             # Reshape the testing data to have 3072 features (32x32x3)


In [ ]:
# convert the labels from integers to vectors
lb = LabelBinarizer()                                   # Create a LabelBinarizer object
trainY = lb.fit_transform(trainY)                       # Transform and binarize the training labels
testY = lb.transform(testY)                             # Transform and binarize the testing labels

# Initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog",
              "frog","horse", "ship", "truck"]

In [ ]:
# Add layers to the model
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="relu"))  # Add a dense layer with 1024 units and ReLU activation function
model.add(Dense(512, activation="relu"))                        # Add another dense layer with 512 units and ReLU activation function
model.add(Dense(10, activation="softmax"))                      # Add a dense layer with 10 units and softmax activation function

In [ ]:
# train the model using SGD
print("[INFO] training network...")
sgd = SGD(0.01)                                               # Initialize SGD optimizer with learning rate 0.01
model.compile(loss="categorical_crossentropy", optimizer=sgd,
              metrics=["accuracy"])  # Compile the model with categorical crossentropy loss and accuracy metric
# Train the model using the training data and validate it using the testing data
H = model.fit(trainX, trainY, validation_data=(testX, testY),
              epochs=10, batch_size=32)  # Train the model for 50 epochs with batch size 32

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32) # Make predictions on the testing data
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames)) # Print classification report

In [ ]:
# Plot the training loss and accuracy
plt.style.use("ggplot")                             # Set the style for the plot
plt.figure()                                        # Create a new figure for the plot
plt.plot(np.arange(0, 10), H.history["loss"], label="train_loss")                # Plot training loss
plt.plot(np.arange(0, 10), H.history["val_loss"], label="val_loss")            # Plot validation loss
plt.plot(np.arange(0, 10), H.history["accuracy"], label="train_acc")          # Plot training accuracy
plt.plot(np.arange(0, 10), H.history["val_accuracy"], label="val_acc")      # Plot validation accuracy
plt.title("Training Loss and Accuracy")                                       # Set the title of the plot
plt.xlabel("Epoch #")                                                        # Label x-axis as "Epoch #"
plt.ylabel("Loss/Accuracy")                                                 # Label y-axis as "Loss/Accuracy"
plt.legend()                                                                # Show legend on the plot